In [53]:
import pandas as pd

In [54]:
# 加载 CSV 文件
data = pd.read_csv("../data/water/整理好的csv/杭州202101-202112/东苕溪202101-202112.csv")

In [55]:
# 处理缺失值
data.fillna(method='ffill', inplace=True)

# 删除重复行
data.drop_duplicates(inplace=True)

/var/folders/2d/fxsd9j2x5c1_4_6qtgkldkmm0000gn/T/ipykernel_17098/3374461071.py:2: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  data.fillna(method='ffill', inplace=True)


In [56]:
# 选择数值列（水温, pH, 溶解氧, 高锰酸钾, 氨氮, 总磷, 总氮, 电导率, 浊度）
numeric_columns = ['水温', 'pH', '溶解氧', '高锰酸钾', '氨氮', '总磷', '总氮', '电导率', '浊度']
data_numeric = data[numeric_columns]

# 数据标准化处理
scaler = StandardScaler()
data_scaled = scaler.fit_transform(data_numeric)

# 转换为Pandas DataFrame格式
data_scaled_df = pd.DataFrame(data_scaled, columns=numeric_columns)

In [57]:
import torch
import torch.nn as nn
import torch.optim as optim

# 定义VAE模型
class VAE(nn.Module):
    def __init__(self, input_dim, hidden_dim, latent_dim):
        super(VAE, self).__init__()
        # 编码器
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.fc2_mu = nn.Linear(hidden_dim, latent_dim)
        self.fc2_logvar = nn.Linear(hidden_dim, latent_dim)
        # 解码器
        self.fc3 = nn.Linear(latent_dim, hidden_dim)
        self.fc4 = nn.Linear(hidden_dim, input_dim)

    def encode(self, x):
        h1 = torch.relu(self.fc1(x))
        return self.fc2_mu(h1), self.fc2_logvar(h1)

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        return mu + eps * std

    def decode(self, z):
        h3 = torch.relu(self.fc3(z))
        return self.fc4(h3)

    def forward(self, x):
        mu, logvar = self.encode(x)
        z = self.reparameterize(mu, logvar)
        return self.decode(z), mu, logvar

# 定义损失函数
def loss_function(recon_x, x, mu, logvar):
    # 使用MSE作为重构损失
    MSE = nn.functional.mse_loss(recon_x, x, reduction='sum')
    # KL散度损失
    KLD = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
    return MSE + KLD

# 训练VAE模型
def train_vae(data, input_dim, hidden_dim=64, latent_dim=16, epochs=100, batch_size=32, learning_rate=1e-3):
    model = VAE(input_dim, hidden_dim, latent_dim)
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    data_tensor = torch.FloatTensor(data)

    dataset = torch.utils.data.TensorDataset(data_tensor)
    dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True)

    for epoch in range(epochs):
        model.train()
        train_loss = 0
        for batch_idx, (inputs,) in enumerate(dataloader):
            optimizer.zero_grad()
            recon_batch, mu, logvar = model(inputs)
            loss = loss_function(recon_batch, inputs, mu, logvar)
            loss.backward()
            train_loss += loss.item()
            optimizer.step()

        print(f'Epoch {epoch+1}, Loss: {train_loss / len(dataloader.dataset)}')

    return model

# 使用VAE模型进行异常检测
def detect_anomalies(model, data, threshold=3):
    data_tensor = torch.FloatTensor(data)
    model.eval()
    with torch.no_grad():
        recon_data, _, _ = model(data_tensor)
        reconstruction_error = torch.mean((recon_data - data_tensor) ** 2, dim=1)
        anomalies = reconstruction_error > threshold
    return anomalies.numpy(), reconstruction_error.numpy()

# 设置参数并训练VAE模型
input_dim = data_scaled_df.shape[1]
vae_model = train_vae(data_scaled_df.values, input_dim, epochs=100)

# 检测异常
anomalies, recon_error = detect_anomalies(vae_model, data_scaled_df.values)

# 将检测结果显示给用户
anomalies_df = data.copy()
anomalies_df['Reconstruction Error'] = recon_error
anomalies_df['Anomaly'] = anomalies

anomalies_df[anomalies_df['Anomaly'] == True]


Epoch 1, Loss: 9.094686742676545
Epoch 2, Loss: 7.7443584062760005
Epoch 3, Loss: 7.24586981502285
Epoch 4, Loss: 6.7919715875373186
Epoch 5, Loss: 6.595417479751814
Epoch 6, Loss: 6.477086843133626
Epoch 7, Loss: 6.30030495950554
Epoch 8, Loss: 6.327385438636774
Epoch 9, Loss: 6.237289374940733
Epoch 10, Loss: 6.1322419622864865
Epoch 11, Loss: 6.142723340398181
Epoch 12, Loss: 5.976163728216615
Epoch 13, Loss: 5.928630604814904
Epoch 14, Loss: 5.883308494287071
Epoch 15, Loss: 5.888238761979523
Epoch 16, Loss: 5.813966247749777
Epoch 17, Loss: 5.826261796704848
Epoch 18, Loss: 5.803587899323825
Epoch 19, Loss: 5.725568112677303
Epoch 20, Loss: 5.733554532402833
Epoch 21, Loss: 5.624817551722635
Epoch 22, Loss: 5.630485707927516
Epoch 23, Loss: 5.58372134310992
Epoch 24, Loss: 5.61069220869263
Epoch 25, Loss: 5.541376620227691
Epoch 26, Loss: 5.513316913984115
Epoch 27, Loss: 5.519613530361382
Epoch 28, Loss: 5.506414373124495
Epoch 29, Loss: 5.482448497225553
Epoch 30, Loss: 5.456335

,省份,城市,河流,流域,断面名称,监测时间,水质类别,水温,pH,溶解氧,高锰酸钾,氨氮,总磷,总氮,电导率,浊度,Reconstruction Error,Anomaly
251,浙江省,杭州市,东苕溪,太湖流域,汪家埠,2021/2/14 12:00,劣Ⅴ,13.13,7.47,8.580,2.399,0.2585,0.5000,2.674,515.2,7.5,4.354386,True
1005,浙江省,杭州市,东苕溪,太湖流域,汪家埠,2021/6/2 8:00,Ⅱ,20.38,7.13,8.569,0.250,0.0544,0.0558,3.588,1.0,1.0,3.339195,True
1103,浙江省,杭州市,东苕溪,太湖流域,奉口,2021/6/11 4:00,劣Ⅴ,27.41,7.45,1.085,5.965,0.1800,0.7836,3.120,268.7,20.7,3.036851,True
1638,浙江省,杭州市,东苕溪,太湖流域,汪家埠,2021/7/31 0:00,Ⅳ,24.53,7.85,6.992,5.885,0.2969,0.2097,4.792,179.8,605.8,3.102592,True


In [73]:
# 加载 CSV 文件
data = pd.read_csv("../data/water/整理好的csv/杭州202101-202112/京杭运河202101-202112.csv")
# 处理缺失值
data.fillna(method='ffill', inplace=True)

# 删除重复行
data.drop_duplicates(inplace=True)
# 选择数值列（水温, pH, 溶解氧, 高锰酸钾, 氨氮, 总磷, 总氮, 电导率, 浊度）
numeric_columns = ['水温', 'pH', '溶解氧', '高锰酸钾', '氨氮', '总磷', '总氮', '电导率', '浊度']
data_numeric = data[numeric_columns]

# 数据标准化处理
scaler = StandardScaler()
data_scaled = scaler.fit_transform(data_numeric)

# 转换为Pandas DataFrame格式
data_scaled_df = pd.DataFrame(data_scaled, columns=numeric_columns)

/var/folders/2d/fxsd9j2x5c1_4_6qtgkldkmm0000gn/T/ipykernel_17098/328221174.py:4: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  data.fillna(method='ffill', inplace=True)


In [74]:
# 定义一个函数用于加载新数据并进行异常检测
def detect_anomalies_on_new_data(model, new_data, threshold=10):
    # 将新数据转化为Tensor
    new_data_tensor = torch.FloatTensor(new_data)
    
    # 使用训练好的模型进行推理
    model.eval()
    with torch.no_grad():
        recon_data, _, _ = model(new_data_tensor)
        reconstruction_error = torch.mean((recon_data - new_data_tensor) ** 2, dim=1)
        anomalies = reconstruction_error > threshold
    
    # 返回检测结果和重构误差
    return anomalies.numpy(), reconstruction_error.numpy()

# 加载新的数据集（确保已经预处理，且维度与原始数据集相同）
new_data_scaled_df = data_scaled_df  # 你需要加载和预处理新的数据

# 使用训练好的模型检测新数据集中的异常
anomalies_new, recon_error_new = detect_anomalies_on_new_data(vae_model, new_data_scaled_df.values)

# 将结果显示给用户
new_anomalies_df = new_data_scaled_df.copy()
new_anomalies_df['Reconstruction Error'] = recon_error_new
new_anomalies_df['Anomaly'] = anomalies_new

# 显示异常的行
data[new_anomalies_df['Anomaly'] == True]

,省份,城市,河流,流域,断面名称,监测时间,水质类别,水温,pH,溶解氧,高锰酸钾,氨氮,总磷,总氮,电导率,浊度
0,浙江省,杭州市,京杭运河,太湖流域,顾家桥,2021/1/1 0:00,Ⅱ,200.00,7.490,9.380,2.20,0.1680,0.0250,1.2400,197.000,25.700
497,浙江省,杭州市,京杭运河,太湖流域,顾家桥,2021/2/17 0:00,Ⅱ,13.08,7.704,7.919,0.48,0.1324,0.0921,1.3141,451.834,2130.447
2797,浙江省,杭州市,京杭运河,太湖流域,顾家桥,2021/9/9 20:00,Ⅲ,27.47,7.676,5.909,0.50,0.0261,0.1033,0.9688,204.095,2288.384
3302,浙江省,杭州市,京杭运河,太湖流域,顾家桥,2021/10/26 8:00,Ⅱ,20.49,8.065,6.946,1.11,0.2312,0.0733,1.4900,228.614,2218.954
3703,浙江省,杭州市,京杭运河,太湖流域,顾家桥,2021/12/1 8:00,Ⅱ,14.81,7.773,9.378,1.82,0.1035,0.0519,2.4041,223.526,2149.161
